# Cours de SQL

Ce cours est présenté sous forme de Notebook, je le mettrai à jour après chaque séance. Il vous suffit de sélectionner une base de donnée et de jouer l'ensemble du notebook pour être prêt à aborder une nouvelle séance.

## Rappels

Nous travaillons avec SQL Server 2019, qui utilise la syntaxe Transact-SQL. En cas de problème/blocage, référez-vous à la documentation de Transact-SQL de préférence. D'autres documentations (MySQL, Postgres, Oracle, PL/SQL) pourraient vous aider, mais il existe des différences entre ces différents dialectes.

## Contexte

Nous allons prendre pour contexte une base de donnée de site de e-commerce assez sommaire. Seule une partie de la base va être modélisée, le but étant de présenter uniquement des notions de base de donnée et non de faire un projet complet.

## Révisions

Pour commencer, nous allons créer une base de données et nous positionner dessus pour travailler.

In [25]:
-- On teste l'existence de la base pour éviter de
-- crasher le notebook si elle existe déjà
IF(DB_ID(N'eCommerce') IS NULL)
    CREATE DATABASE [eCommerce];
GO
USE [eCommerce];
GO

Maintenant que nous avons une base de données, nous allons pouvoir créer une table. Nous allons commencer par la table des produits. Pour le moment, nous produits auront un `Id` ainsi qu'un `Nom`.

Afin de faire des colones avec un auto-incrément, nous allons utiliser la propriété `Identity`. Nous allons également positionner une clé primaire sur notre colone `Id`.

In [26]:
IF NOT EXISTS (
    SELECT  [name]
    FROM    [sysobjects]
    WHERE   [name]='Produit_PRD'
            AND [xtype]='U')
    CREATE TABLE [dbo].[Produit_PRD] (
        [Id] BIGINT IDENTITY(1, 1) PRIMARY KEY,
        [Nom] NVARCHAR(255) NOT NULL,
    )
    GO

Nous allons également ajouter une table qui contiendra des revues sur les différents articles proposés sur le site. Les revues auront une `Id`, clé primaire identité, une clé étrangère vers la table des produits, un titre et un contenu.

In [27]:
IF NOT EXISTS (
    SELECT  [name]
    FROM    [sysobjects]
    WHERE   [name]='Revue_REV'
            AND [xtype]='U')
    CREATE TABLE [dbo].[Revue_REV] (
        [Id] BIGINT IDENTITY(1, 1) PRIMARY KEY,
        [PRD_Id] BIGINT NOT NULL,
        [Titre] NVARCHAR(255) NOT NULL,
        [Contenu] NVARCHAR(MAX) NOT NULL,
        CONSTRAINT fk_PRD_Id_REV
            FOREIGN KEY ([PRD_Id]) REFERENCES [Produit_PRD]([Id])
    )
GO

### Opérations de base

Maintenant que nous avons des tables, nous allons pouvoir y mettre du contenu, le récupérer, le modifier, le supprimer.

Pour l'insertion, nous ne devons pas saisir de valeur pour les colonnes identités. Ainsi, si je souhaite insérer une ligne dans ma table de produit, je pourrais faire la requête suivante :

In [28]:
/*
INSERT INTO [dbo].[Produit_PRD] ([Nom])
VALUES (N'iPone 11'), (N'One Plus 7')
GO
*/

On peut récupérer les données avec l'instruction `SELECT`

In [29]:
SELECT
        [Id],
        [Nom]
FROM    [dbo].[Produit_PRD]
GO

On va pouvoir faire des mises à jour à l'aide de la clause `UPDATE`

In [30]:
/*
UPDATE [dbo].[Produit_PRD]
SET [Nom] = N'iPhone 11'
WHERE [Id] = 1
GO
*/

On peut également supprimer des lignes avec l'instruction `DELETE`

In [31]:
/*
DELETE FROM [dbo].[Produit_PRD] WHERE [Id] = 2
GO
*/

Nous pouvons modifier la structure des table à postériori, à l'aide de l'instruction `ALTER TABLE`. Nous allons modifier notre table de produits pour ajouter une colone avec le prix, un descriptif et un contenu.

In [32]:
IF NOT EXISTS (
    SELECT [name] 
    FROM   [sys].[columns]
    WHERE  [object_id] = OBJECT_ID(N'[dbo].[Produit_PRD]') 
            AND [name] = 'Prix'
)
    ALTER TABLE [dbo].[Produit_PRD]
    ADD [Prix] DECIMAL(5, 2) NOT NULL,
        [Descriptif] NVARCHAR(255) NULL,
        [Contenu] NVARCHAR(MAX) NULL,
        DEFAULT 0 FOR [Prix]
GO

### Jointures

Nous pouvons faire des jointures entre le contenu de différentes tables. Il existe différents types de jointures :

![Jointures SQL](https://i.pinimg.com/originals/78/06/27/78062746cee62bdc112aad0582ae49c4.jpg)

Ceci va nous permettre de créer des requêtes complexes.

In [33]:
SELECT
    [PRD].[Id] AS [ProduitId],
    [PRD].[Nom] AS [ProduitNom],
    [REV].[Titre] AS [RevueTitre]
FROM [dbo].[Produit_PRD] AS [PRD]
INNER JOIN [dbo].[Revue_REV] AS [REV]
    ON [REV].[PRD_Id] = [PRD].[Id]
ORDER BY [PRD].[Id] ASC
GO

### Vues

Nous pouvons utiliser des vues pour avoir une vision de plusieurs tables aglomérées et mise en cache par le SGBD.

In [34]:
CREATE OR ALTER VIEW [dbo].[VueRevues]
AS
    SELECT
        [PRD].[Id] AS [ProduitId],
        [PRD].[Nom] AS [ProduitNom],
        [REV].[Titre] AS [RevueTitre]
    FROM [dbo].[Produit_PRD] AS [PRD] (NOLOCK)
    INNER JOIN [dbo].[Revue_REV] AS [REV] (NOLOCK)
        ON [REV].[PRD_Id] = [PRD].[Id]
GO

In [35]:
SELECT [ProduitId], [ProduitNom], [RevueTitre]
FROM [dbo].[VueRevues]
GO

## Fonctions

En SQL, nous pouvons créer différents types de fonctions. Nous en étudierons deux :
- Les fonctions table
- Les fonctions scalaires

Les fonctions tables sont des fonctions qui vont nous revoyer des données structurées sous forme d'une table.

In [36]:
CREATE OR ALTER FUNCTION [dbo].[GetRevues](@PrdId BIGINT)
RETURNS TABLE
AS
RETURN
(
    SELECT [Titre], [Contenu]
    FROM [dbo].[Revue_REV] (NOLOCK)
    WHERE [PRD_Id] = @PrdId
)
GO

In [37]:
SELECT [Titre], [Contenu] FROM [dbo].[GetRevues](1)
GO

Nous avons également les fonctions scalaires, qui vont retourner une valeur. SQL Server en propose plusieurs en natif, voici quelques exemples :

In [38]:
SELECT GETDATE() AS [Date], [Titre] FROM [Revue_REV]
GO

In [39]:
SELECT
    COALESCE([Descriptif], [Contenu]) AS [Resultat],
    [Descriptif],
    [Contenu]
FROM [dbo].[Produit_PRD]
GO

On va également pouvoir définir nos propres fonctions

In [40]:
CREATE OR ALTER FUNCTION [dbo].[CalculPromo](@Prix DECIMAL(5, 2))
RETURNS DECIMAL(5, 2)
AS
BEGIN
    DECLARE @Taux DECIMAL(5, 2);

    IF (@Prix > 50)
        SET @Taux = 0.65;
    ELSE
        SET @Taux = 0.75;

    RETURN(@Prix * @Taux)
END
GO

In [43]:
SELECT [Prix], [dbo].[CalculPromo]([Prix]) AS [Promo] FROM [Produit_PRD]
GO

## Triggers et curseurs

Avant d'entrer dans le vif du sujet, nous allons modifier notre table produits pour y ajouter deux colonnes :
- Une pour la date d'ajout de produit
- Une pour la date de mise à jour du produit

In [45]:
IF NOT EXISTS (
    SELECT [name] 
    FROM   [sys].[columns]
    WHERE  [object_id] = OBJECT_ID(N'[dbo].[Produit_PRD]') 
            AND [name] = 'Ajout'
)
    ALTER TABLE [dbo].[Produit_PRD]
    ADD [Ajout] DATETIME2 NOT NULL,
        [Modification] DATETIME2 NULL,
        DEFAULT GETDATE() FOR [Ajout]
GO

Commands completed successfully.

Total execution time: 00:00:00.002

Notre table de produit comporte désormais deux champs supplémentaire, une date avec une valeur par défaut (date d'ajout) et une date sans valeur par défaut (date de dernière modification).

Si j'ai envie de maintenir à jour la date de dernière modification de ma table sans devoir veiller à mettre la valeur correctement à chaque fois, je peux définir un trigger lors de l'update de ma (ou mes) ligne(s).

Pour ce faire, nous allons également utiliser un curseur pour itérer sur la (ou les) valeur(s) modifiée(s).

In [46]:
CREATE OR ALTER TRIGGER [dbo].[trig_UpdatePrdModification]
   ON  [dbo].[Produit_PRD] 
   AFTER UPDATE
AS 
BEGIN
	SET NOCOUNT ON;

    DECLARE @num INT
	DECLARE @PrdId INT

	SELECT @num = COUNT(*) from INSERTED

	IF @num > 1
	BEGIN
		DECLARE curs CURSOR FOR SELECT [Id] from INSERTED
		OPEN curs		
		FETCH NEXT FROM curs into @PrdId; 
		WHILE (@@FETCH_STATUS = 0)
		BEGIN
			UPDATE	[dbo].[Produit_PRD]
			SET		[Modification] = GETDATE()
			WHERE	[Id] = @PrdId

			FETCH NEXT FROM curs into @PrdId; 
		END

		CLOSE curs
		DEALLOCATE curs
	END
	ELSE
	BEGIN
		SELECT TOP 1 @PrdId = [Id] from INSERTED

		UPDATE  [dbo].[Produit_PRD]
        SET		[Modification] = GETDATE()
        WHERE	[Id] = @PrdId
	END
END

Commands completed successfully.

Total execution time: 00:00:00.032